<a href="https://colab.research.google.com/github/Kepners/ChopOnions/blob/main/PieTrends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install requests pandas


In [5]:
import requests
import json
import pandas as pd
from io import StringIO
from time import sleep

class GoogleTrendsScraper:

    def __init__(self):
        self.session = requests.Session()
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

    def _get_json(self, url):
        """Helper function to fetch and parse JSON response."""
        response = self.session.get(url, headers=self.headers)
        if response.status_code == 200:
            try:
                data = json.loads(response.text[5:])  # Remove leading non-JSON chars if necessary
                return data
            except json.JSONDecodeError as e:
                print("JSON decoding error:", e)
                return None
        else:
            print("Error fetching data:", response.status_code)
            return None

    def get_trending_searches(self, region='US'):
        """Fetch daily trending searches for a specific region."""
        url = f"https://trends.google.com/trends/api/dailytrends?geo={region}"
        data = self._get_json(url)
        if data:
            return data['default']['trendingSearchesDays']
        else:
            return None

    def get_interest_over_time(self, keyword, timeframe='now 7-d', geo=''):
        """Fetch interest over time for a keyword within a specified timeframe."""
        url = f"https://trends.google.com/trends/api/widgetdata/multiline/csv?geo={geo}&q={keyword}&time={timeframe}"
        response = self.session.get(url, headers=self.headers)

        if response.status_code == 200:
            try:
                # Parsing CSV from text to DataFrame
                csv_data = pd.read_csv(StringIO(response.text))
                return csv_data
            except Exception as e:
                print("Error parsing CSV:", e)
                return None
        else:
            print("Error fetching interest over time:", response.status_code)
            return None

    def get_interest_by_region(self, keyword, geo='US'):
        """Fetch interest by region for a keyword."""
        url = f"https://trends.google.com/trends/api/widgetdata/comparedgeo?geo={geo}&q={keyword}"
        data = self._get_json(url)
        if data:
            return data['default']['geoMapData']
        else:
            return None

    def get_related_queries(self, keyword, geo=''):
        """Fetch related queries for a keyword."""
        url = f"https://trends.google.com/trends/api/widgetdata/relatedsearches?geo={geo}&q={keyword}"
        data = self._get_json(url)
        if data:
            return data['default']['rankedList']
        else:
            return None

    def _rate_limit(self, delay=2):
        """Simple rate-limiting function."""
        print("Sleeping for", delay, "seconds to avoid rate limits.")
        sleep(delay)
